In [1]:
from bs4 import *
import requests as rp
import os
from msedge.selenium_tools import *
import time
from tqdm import tqdm
import pickle
import numpy as np
import psycopg2 as ps
import random
from selenium.webdriver.common.by import *
import selenium
import ast
import concurrent.futures
import threading
import tensorflow as tf
import json

options = EdgeOptions()
options.use_chromium = True
options.add_argument("headless")
options.add_argument("disable-gpu")

In [7]:
def push_to_table(fname, lname, gender, npi, speciality, email,country,state,city,verify,password,degree,university,year,fee,bio,experience):
    conn = ps.connect(
        host = '127.0.0.1',
        database = 'zocdoc',
        user = 'postgres',
        password = 'ss123546',
    )
    cur = conn.cursor()
    cur.execute('insert into "Practo" values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', (fname, lname, gender, npi, speciality, email,country,state,city,verify,password,degree,university,year,fee,bio,experience))
    conn.commit()
    conn.close() 

In [5]:
#data = []
def get_urls(url):
    driver = Edge(options = options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    page_urls = []
    out = soup.find_all('div', {'class':'info-section'})
    for i in out:
        links = i.find('a')['href']
        data.append(links)
        
    driver.close()
    threadLimiter.release()

In [4]:
threadLimiter = threading.BoundedSemaphore(50)
def url_collector(city = 'delhi'):
    for i in tqdm(range(100,852)):#851
        url = "https://www.practo.com/search?results_type=doctor&q=%5B%7B%22word%22%3A%22doctor%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22type%22%7D%5D&city="+city+"&page="+str(i)
        t= threading.Thread(target = get_urls, args = [url])
        threadLimiter.acquire()
        try:
            t.start()
        except:
            pass

In [6]:
get_urls = url_collector()
pickle_out = open('checkpoint_practo_url_delhi.pickle','wb')#change for another checkpoint
pickle.dump(data, pickle_out)
pickle_out.close()

 61%|█████████████████████████████████████████████████                               | 461/752 [05:10<00:51,  5.60it/s]Exception in thread Thread-420:
Traceback (most recent call last):
  File "E:\software\anaconda\envs\gputest\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\software\anaconda\envs\gputest\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-1b586a6f15a1>", line 4, in get_urls
    driver.get(url)
  File "E:\software\anaconda\envs\gputest\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "E:\software\anaconda\envs\gputest\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "E:\software\anaconda\envs\gputest\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, scre

In [8]:
errors = []
def get_profile(url):
    driver = Edge(options = options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    out = {}
    try:
        name = soup.find('h1', {'data-qa-id':'doctor-name'}).text

        #first and last name
        name = name.split(' ')
        if len(name) == 2 and name[0] == 'Dr.':
            fname = name[1]
            lname = ''
        elif name[0] != 'Dr.':
            if len(name) == 1:
                fname = name[0]
                lname = ''
            else:
                fname = name[0]
                lname = ' '.join(name[1:])
        else:
            fname = name[1]
            lname = ' '.join(name[2:])

        #gender
        gender = get_gender(fname)

        #npi
        reg_details = soup.find('div', {'data-qa-id':'registrations-item'}).text
        

        #specility
        speciality = soup.find('div', {'data-qa-id':'specializations-item'}).text

        #email
        email = ('.'.join(name[1:])+'@gmail.com').lower()

        #country,state,city
        country = 'India'
        State = "Delhi"
        city = "Delhi"

        #password verify
        password = 'DemoDoc@1234'
        verify = 'True'

        #fee,bio,experience
        try:
            fee = soup.find('span', {'data-qa-id':'consultation_fee'}).text[1:]
        except:
            fee = random.choice([100,150, 200,400,250])  
        
        driver.find_element_by_class_name("u-t-link").click()
        bio = ' '.join([i.text for i in driver.find_elements_by_class_name('c-profile__description')])[:-8]
        experience = soup.find('div', {'data-qa-id':'doctor-specializations'}).find_all('h2')[-1].text.split("\xa0")[0]
        
            
        #degree, university,year
        education = []
        dat = [i.text for i in soup.find_all('div', {'data-qa-id':'educations-item'})]
        for i in dat:
            m = i.split(', ')
            education.append(
                {
                    'year':m[1],
                    'degree':m[0].split(' - ')[0],
                    'university':' '.join(m[0].split(' - ')[1:])
                }            
            )
        push_to_table(fname, lname, gender, str(reg_details), speciality, email,country,State,city,verify,password,str(education),str(education),url,str(fee),bio,str(experience))
    except:
        errors.append(url)
    finally:
        driver.close()
        threadLimiter.release()

In [ ]:
threadLimiter = threading.BoundedSemaphore(50)
future = []
count = 0
for i in tqdm(urls[7311:]):
    count+=1
    t= threading.Thread(target = get_profile, args = [str('https://www.practo.com'+i)])
    threadLimiter.acquire()
    t.start()
    '''future.append(t)
    if((count)%1000 == 0):
        for thread in future:
            thread.join()
        future = []
for thread in future:
    thread.join()
    future = []'''

 87%|████████████████████████████████████████████████████████████████████▏         | 1014/1159 [19:25<19:28,  8.06s/it]Exception in thread Thread-1149:
Traceback (most recent call last):
  File "E:\software\anaconda\envs\gputest\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\software\anaconda\envs\gputest\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-4ce93f29013c>", line 75, in get_profile
    driver.close()
  File "E:\software\anaconda\envs\gputest\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 688, in close
    self.execute(Command.CLOSE)
  File "E:\software\anaconda\envs\gputest\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "E:\software\anaconda\envs\gputest\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, st

In [21]:
pickle_out = open('checkpoint_practo_url_delhi.pickle','wb')#change for another checkpoint
pickle.dump(data, pickle_out)
pickle_out.close()

NameError: name 'error' is not defined

In [10]:
pickle_in = open('checkpoint_practo_url_delhi.pickle','rb')
data = pickle.load(pickle_in)

In [11]:
len(data)

8470

In [12]:
urls = data 

In [10]:
urls += k

In [22]:
#clearing error

pickle_out = open('checkpoint_practo_url_delhi_error.pickle','wb')#change for another checkpoint
pickle.dump(errors, pickle_out)
pickle_out.close()

In [ ]:
import os
os.system('shutdown /s /t 1')

In [9]:
pickle_in = open('checkpoint_practo_url_delhi_error.pickle','rb')
k = pickle.load(pickle_in)
len(k)

121

In [9]:
model = tf.keras.models.load_model('gender_prediction_name.model')
token_table = {}
j = 0
for i in 'abcdefghijklmnopqrstuvwxyz':
    token_table[i] = j/25
    j+=1
def get_gender(name):
    name = [token_table[i.lower()] for i in list(name)]
    curr_len = len(name)
    padd_len = 15 - curr_len
    name += [-1.0]*padd_len
    if np.argmax(model.predict([name])) == 0:
        return('Female')
    else:
        return('Male')

In [15]:
"https://v2.namsor.com/NamSorAPIv2/api2/json/country/souvik -H accept: application/json -H X-API-KEY: e1a21678aea7c569bb0059523e41fbf6"

'https://v2.namsor.com/NamSorAPIv2/api2/json/country/souvik -H accept: application/json -H X-API-KEY: e1a21678aea7c569bb0059523e41fbf6'

In [12]:
get_gender('souvik')

'Male'

# POSTMAN

In [3]:
import pickle
pickle_in = open('inserted_practo_postman.pickle','rb')
m = pickle.load(pickle_in)
len(m)

208

In [22]:
r = rp.get('https://server.docplus.online/doctors/getDoc/603745067fcc6c59ed6631ee')

In [3]:
def retrive_data():
    conn = ps.connect(
        host = '127.0.0.1',
        database = 'zocdoc',
        user = 'postgres',
        password = 'ss123546',
    )
    cur = conn.cursor()
    cur.execute('select * from "Practo"')
    data = cur.fetchall()
    return data

In [4]:
pushed_data = retrive_data()

In [43]:
wrong = []
something_wrong = []
m = 0
def send_data(out):
    #inserting name
    first_name = out[0]
    last_name = out[1]


    #inserting gender
    gender = out[2]

    #inserting npi
    npi = out[3].split(' ')[0]

    #inserting speciality
    speciality = out[4]

    #inserting about
    about = out[15]

    #inserting email
    email = out[5]

    #inserting address
    country = out[6]
    city = out[7]
    state = out[8]

    #inserting password
    password = out[10]
    verify = out[9]

    #inserting fees
    fee = out[14]

    #education
    edu = ast.literal_eval(out[11])

    #register doctors
    payload = {'firstName':first_name,
              'lastName':last_name,
              'email':email,
              'country':country,
              'state':state,
              'city':city,
              'password':password,
              'basic':{},
              'specialty':speciality,
              'registration_number':npi}

    r = rp.post('https://server.docplus.online/doctors/register', json = payload)
    get_id = r.json()['data']['_id'] #id of created
    datas[email] = get_id

    if(len(datas.keys()) % 100 == 0):
        pickle_out = open('inserted_practo_postman1.pickle','wb')#change for another checkpoint
        pickle.dump(datas, pickle_out)
        pickle_out.close()


    #update_doctors
    payload = {
        'gender':gender,
        'id':get_id,
        'bio':about,
        'education': edu,
        'fee':fee,
        'verify':'true',
    }
    r = rp.post('https://server.docplus.online/doctors/profile/update', json = payload)

'''except:
something_wrong.append(out)
print('something wrong', r.text)
return -1
except:
print('error')
wrong.append(out)'''

"except:\nsomething_wrong.append(out)\nprint('something wrong', r.text)\nreturn -1\nexcept:\nprint('error')\nwrong.append(out)"

In [44]:
for i in tqdm(pushed_data[7:8]):
    m = send_data(i)
    if m == -1:
        break

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


KeyError: 'data'

In [3]:
pickle_out = open('inserted_practo_postman2.pickle','wb')#change for another checkpoint
pickle.dump(datas, pickle_out)

NameError: name 'datas' is not defined

In [3]:
r = rp.get('https://server.docplus.online/doctors/getDoc/60423c5d2198a1f6905f5e77')

In [34]:
b = ['_id', 'npi', 'email', 'password', 'specialty', 'customerProfile', 'city', 'state', 'country', 'firstName', 'lastName', 'referralId', 'createdOn', 'education']
for i in b: print(i, r.json()['data'][i])

_id 60423c5d2198a1f6905f5e77
npi 22644
email arun.sarkar@gmail.com
password 985c201c0983d138a856ffb0c8d53890
specialty General Physician
customerProfile cus_J3lwcdQEzmgKxS
city Jharkhand
state Ranchi
country India
firstName Arun
lastName Sarkar
referralId ArunXdp2N_d
createdOn 2021-03-05T14:12:45.315Z
education [{'_id': '604241022198a1f6905f5ec1', 'year': 1985, 'degree': 'MBBS', 'university': 'Ranchi University'}, {'_id': '604241022198a1f6905f5ec2', 'year': 1998, 'degree': 'MD', 'university': 'General Medicine Ranchi University'}]


In [9]:
folder_path = "E:/Youtube game/works/Ranchi/Ranchi/"
dic = {}
for i in os.listdir(folder_path):
    dic[" ".join(i.split()[1:]).split(".")[0]] = folder_path + i

In [23]:
jaipur = pushed_data[207:2864]

In [6]:
jaipur_filter = []
for i in ranchi:
    if(i[8] == 'Ranchi'):
        ja_filter.append(i)

In [ ]:
for i in ranchi_filter:
    name = i[0]+" "+i[1]
    try: print(dic[name])
    except: print("image_not_available" +" "+i[-4])

In [10]:
DIR = 'E:/Youtube game/works/Ranchi/Ranchi/'
images_name = [i for i in os.listdir(DIR)]
img = {}
for i in images_name:
    img[i] = DIR+i
    
    
for out in tqdm(ranchi_filter[20:]):
    key = out[3]+'.jpg'
    
    if(key in images_name):
    
        first_name = out[0]
        last_name = out[1]

        #inserting gender
        try:
            gender = out[2].lower()
        except:
            gender = "male"

        #inserting npi
        npi = out[3].split(' ')[0]

        #inserting speciality
        speciality = out[4]

        #inserting about
        about = out[15]

        #inserting email
        email = out[5]

        #inserting address
        country = out[6]
        city = out[7]
        state = out[8]

        #inserting password
        password = out[10]
        verify = out[9]

        #inserting fees
        fee = out[14]

        #education
        edu = ast.literal_eval(out[11])
        for i in edu:
            try:
                i['year'] = int(i['year'])
            except:
                pass
            if type(i['year']) != int:
                i['year'] = ''

        payload = {'firstName':first_name,
                  'lastName':last_name,
                  'email':email,
                  'country':country,
                  'state':state,
                  'city':city,
                  'password':password,
                  'basic':{},
                  'specialty':speciality,
                  'registration_number':npi}

        r = rp.post('https://server.docplus.online/doctors/register', json = payload)

        if r.json()['status'] == True:
            r.json()['data']
            get_id = r.json()['data']['_id'] #id of created
            print(get_id, email)
            m[email] = get_id


            payload = {
                            'gender':gender,
                            'id':m[email],
                            'bio':about,
                            'education': edu,
                            'fee':fee,
                            'verify':'true',
                        }
            r = rp.post('https://server.docplus.online/doctors/profile/update', json = payload)
            print(r.text)
            
            
            payload = {
                'id':m[email],
            }
            files = [
                ('image',(key, open(img[key],'rb'),'image/jpg'))
            ]
            r = rp.request("POST",'https://server.docplus.online/doctors/upload/image', data = payload, files=files)
            print(r.json())
            
        else:
            print(r.json()['message'])
            continue
    else:
        print("Not present", out[0])


  0%|                                                                                          | 0/171 [00:00<?, ?it/s]

606ad4a60030edda327b7a6f rakesh.mishra@gmail.com
{"status":true,"message":"Profile successfully updated","data":{"basic":{"organization_name":"not found","organizational_subpart":"not found","enumeration_date":"not found","last_updated":"not found","status":"not found","telephone_number":"not found","title_or_position":"not found","name_prefix":"not found","name_suffix":"not found","sole_proprietor":"not found","gender":"not found","name":"Rakesh Mishra"},"document":{"idProof":false,"registrationProof":false,"establishment":false},"address":[],"taxonomies":[],"practiceLocation":[],"diseases":[],"description":"NA","steps":[0,0,0,0],"is_in_network":true,"is_superDoc":false,"location_ids":[],"isActive":true,"isApproved":false,"availability":[],"appointments":[],"picture":[],"profileStatus":true,"question":[],"appointmentsString":"{\"duration\":\"15\",\"id\":\"\",\"weekdaysArr\":[{\"days\":[\"wednesday\",\"thursday\",\"friday\"],\"startTime\":\"02:30\",\"endTime\":\"11:30\",\"lunchStart\":


  1%|▍                                                                                 | 1/171 [00:12<34:59, 12.35s/it]

{'status': True, 'message': 'Image uploaded successfully'}
606ad4b40030edda327b7a8e sathi.rani.sarkar@gmail.com
{"status":true,"message":"Profile successfully updated","data":{"basic":{"organization_name":"not found","organizational_subpart":"not found","enumeration_date":"not found","last_updated":"not found","status":"not found","telephone_number":"not found","title_or_position":"not found","name_prefix":"not found","name_suffix":"not found","sole_proprietor":"not found","gender":"not found","name":"Sathi Rani Sarkar"},"document":{"idProof":false,"registrationProof":false,"establishment":false},"address":[],"taxonomies":[],"practiceLocation":[],"diseases":[],"description":"NA","steps":[0,0,0,0],"is_in_network":true,"is_superDoc":false,"location_ids":[],"isActive":true,"isApproved":false,"availability":[],"appointments":[],"picture":[],"profileStatus":true,"question":[],"appointmentsString":"{\"duration\":\"15\",\"id\":\"\",\"weekdaysArr\":[{\"days\":[\"wednesday\",\"thursday\",\"frid


  1%|▉                                                                                 | 2/171 [00:25<36:47, 13.06s/it]

...

{'status': True, 'message': 'Image uploaded successfully'}
Not present Spandana
Not present Ujjwal
Not present Ankur
Not present Mr.
606ad89e0030edda327b8659 jitendra.kumar@gmail.com
{"status":true,"message":"Profile successfully updated","data":{"basic":{"organization_name":"not found","organizational_subpart":"not found","enumeration_date":"not found","last_updated":"not found","status":"not found","telephone_number":"not found","title_or_position":"not found","name_prefix":"not found","name_suffix":"not found","sole_proprietor":"not found","gender":"not found","name":"Jitendra Kumar"},"document":{"idProof":false,"registrationProof":false,"establishment":false},"address":[],"taxonomies":[],"practiceLocation":[],"diseases":[],"description":"NA","steps":[0,0,0,0],"is_in_network":true,"is_superDoc":false,"location_ids":[],"isActive":true,"isApproved":false,"availability":[],"appointments":[],"picture":[],"profileStatus":true,"question":[],"appointmentsString":"{\"duration\":\"15\",\"id\


 96%|████████████████████████████████████████████████████████████████████████████▋   | 164/171 [17:03<00:14,  2.08s/it]

{'status': True, 'message': 'Image uploaded successfully'}
Not present Sarita
Not present Suhagini
606ad8a40030edda327b8678 lt.col.vivek.kumar.singh@gmail.com
{"status":true,"message":"Profile successfully updated","data":{"basic":{"organization_name":"not found","organizational_subpart":"not found","enumeration_date":"not found","last_updated":"not found","status":"not found","telephone_number":"not found","title_or_position":"not found","name_prefix":"not found","name_suffix":"not found","sole_proprietor":"not found","gender":"not found","name":"Lt Col Vivek Kumar Singh"},"document":{"idProof":false,"registrationProof":false,"establishment":false},"address":[],"taxonomies":[],"practiceLocation":[],"diseases":[],"description":"NA","steps":[0,0,0,0],"is_in_network":true,"is_superDoc":false,"location_ids":[],"isActive":true,"isApproved":false,"availability":[],"appointments":[],"picture":[],"profileStatus":true,"question":[],"appointmentsString":"{\"duration\":\"15\",\"id\":\"\",\"weekd

100%|████████████████████████████████████████████████████████████████████████████████| 171/171 [17:09<00:00,  6.02s/it]

{'status': True, 'message': 'Image uploaded successfully'}
Not present Sakshi
Not present Prashant
Not present Prashant
Not present S


In [11]:
m

{'amitesh.anand@gmail.com': '6065f3453b8e22d55b2daf52',
 'lalit.kongari@gmail.com': '6065f34c3b8e22d55b2daf64',
 'aftab.alam.ansari@gmail.com': '6065f3523b8e22d55b2daf75',
 'thounaojam.momon.singh@gmail.com': '6065f3583b8e22d55b2daf86',
 'rashmi@gmail.com': '606602fa1c41e5717873b776',
 'kumar.sourabh@gmail.com': '606603001c41e5717873b786',
 'kumar.rajesh.chandra.ojha@gmail.com': '606603061c41e5717873b798',
 'ratnesh.kumar@gmail.com': '6066031b1c41e5717873b7a8',
 'sarbjaya.singh@gmail.com': '606603221c41e5717873b7b9',
 'nitisha.lath@gmail.com': '606ad43a0030edda327b799a',
 'virul.shrivastava@gmail.com': '606ad4460030edda327b79b9',
 'saha.piyali@gmail.com': '606ad4510030edda327b79d8',
 'rajesh.kumar.singh@gmail.com': '606ad45a0030edda327b79f6',
 'm..c..khaitan@gmail.com': '606ad4670030edda327b7a15',
 'sunil.kumar.singh@gmail.com': '606ad4760030edda327b7a33',
 'rajiv.kumar@gmail.com': '606ad4820030edda327b7a51',
 'rakesh.mishra@gmail.com': '606ad4a60030edda327b7a6f',
 'sathi.rani.sarkar@g

In [43]:
payload = {
    'id':'6065e1b13b8e22d55b2daf2f',
}
files = [
    ('image',(key, open(img[key],'rb'),'image/jpg'))
]
r = rp.request("POST",'https://server.docplus.online/doctors/upload/image', data = payload, files=files)

In [10]:
r.text

'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx/1.18.0 (Ubuntu)</center>\r\n</body>\r\n</html>\r\n'

In [18]:
pickle_out = open('inserted_practo_postman.pickle','wb')#change for another checkpoint
pickle.dump(m, pickle_out)
pickle_out.close()

In [19]:
pickle_in = open('inserted_practo_postman.pickle','rb')
outcome = pickle.load(pickle_in)

In [21]:
len(outcome)

117

In [123]:
r.text

'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx/1.18.0 (Ubuntu)</center>\r\n</body>\r\n</html>\r\n'

In [17]:
img[key]

'E:/Youtube game/works/Ranchi/Ranchi/19088A Karnataka State Dental Council, 2006.jpg'

In [15]:
len(outcome)

471

In [17]:
len(m)

1407

In [32]:
import pandas as pd

In [42]:
email = []
npi = []
for i,j in m.items():
    email.append(i)
    npi.append(j)
    
convertion_data = {
    'email':email,
    'id':npi
}

df = pd.DataFrame.from_dict(convertion_data)

In [44]:
df.to_excel(r'C:\Users\Souvik_2\Desktop\practo_postman_doctors.xlsx')

In [9]:
import concurrent.futures
import time

In [24]:
a = [1]
def printf():
    a[0]+=1
    return([a[0]])
    time.sleep(10)

In [47]:
for _ in range(100):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.map(printf)


In [48]:
for i in range(100):
    t1 = threading.Thread(target = printf)
    t1.start()

In [21]:
driver = Edge(options = options)
driver.get("https://www.practo.com/search?results_type=doctor&q=%5B%7B%22word%22%3A%22doctor%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22type%22%7D%5D&city=korba&page=1")
soup = BeautifulSoup(driver.page_source, 'html.parser')
m = soup.find('div', {'data-qa-id':'pagination_container'})
print(m)
data = []
for i in m.find_all('li'):
       data.append(i.text)
print(data)

<div class="pure-u-14-24" data-qa-id="pagination_container"></div>
[]


In [15]:
import threading
data = []
threadLimiter = threading.BoundedSemaphore(50)
def get_urls(url):
    all_urls = []
    driver = Edge(options = options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    out = soup.find_all('div', {'class':'info-section'})
    for i in out:
        links = i.find('a')['href']
        data.append(links)
        all_urls.append(links)
    driver.close()
        
for i in tqdm(range(100,135)):
    future = []
    url = "https://www.practo.com/search?results_type=doctor&q=%5B%7B%22word%22%3A%22doctor%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22type%22%7D%5D&city=jaipur&page="+str(i)
    t= threading.Thread(target = get_urls, args = [url])
    threadLimiter.acquire()
    try:
        t.start()
    except:
        threadLimiter.release()
        t.terminate()
    future.append(t)
for thread in future:
    thread.join()
        

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:02<00:00, 13.81it/s]


In [7]:
len(data)

490

In [19]:
pickle_out = open('jaipur_urls.pickle','wb')#change for another checkpoint
pickle.dump(data, pickle_out)

In [4]:
pickle_in  = open('jaipur_urls.pickle','rb')
urls = pickle.load(pickle_in)

In [8]:
len(urls)

1310

In [17]:
data+=pickle_out

In [33]:
a = []
for i in pushed_data:
    if(i[7] == 'Delhi'):
        a.append(i)

In [34]:
len(a)

6522

In [14]:
a = [i[3].split()[0] for i in a]

In [25]:
b = []
for i in pushed_data:
    if(i[7] == 'Delhi'):
        b.append(i)

In [30]:
len(b)

6514

In [29]:
6334 - 6288

46

# Image collection

In [27]:
for i in jaipur[:10]:
    print(i[])

[('Rishav',
  'Bansal',
  'male',
  'DMC/R/9919 Delhi Medical Council, 2014',
  'Geriatrician',
  'rishav.bansal@gmail.com',
  'India',
  'Jharkhand',
  'Ranchi',
  'True',
  'DemoDoc@1234',
  "[{'year': 'New Delhi', 'degree': 'MBBS', 'university': 'All India Institute of Medical Sciences'}, {'year': 'New Delhi', 'degree': 'MD', 'university': 'Geriatrics All India Institute of Medical Sciences'}]",
  "[{'year': 'New Delhi', 'degree': 'MBBS', 'university': 'All India Institute of Medical Sciences'}, {'year': 'New Delhi', 'degree': 'MD', 'university': 'Geriatrics All India Institute of Medical Sciences'}]",
  'https://www.practo.com/Jaipur/doctor/dr-rishav-bansal-geriatrician?specialization=geriatrician',
  '400',
  "Dr. Rishav Bansal is Consultant and Head of department, Geriatric Medicine at Santokba Durlabhji Memorial Hospital (SDMH), Jaipur. He is a Geriatrician/Senior Citizen Specialist Physician. Geriatric Medicine is a specialty of Medicine that deals exclusively with the “healthc

In [26]:
#image_collector
import urllib.request
import time
import os
threadLimiter = threading.BoundedSemaphore(10)
def get_image(url):
    driver = Edge(options = options)
    driver.get(url[0])
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    image = soup.find('img', {'class':'c-profile__image'})['src']
    save_image(image, url[1])
    threadLimiter.release()
        
def image_collector(urls):
    for i in tqdm(urls):
        t= threading.Thread(target = get_image, args = [[i[-4], i[3]]])
        threadLimiter.acquire()
        t.start()
        
def save_image(url, file_name, file_path = 'E:/Youtube game/works/Jaipur/'):
    try:
        full_path = file_path+file_name+'.jpg'
        urllib.request.urlretrieve(url, full_path)
    except:
        print('error', file_name, url)
        pass
       
image_collector(jaipur[0:10])#i[-4], i[3]

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1670.77it/s]


error DMC/R/9919 Delhi Medical Council, 2014 https://imagesx.practo.com/providers/b097bbe7-73e7-48c9-be14-262642d1ede7.jpg?i_type=t_100x100


Exception in thread Thread-15:
Traceback (most recent call last):
  File "E:\software\anaconda\envs\gputest\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\software\anaconda\envs\gputest\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-26-dc3b696db679>", line 10, in get_image
    image = soup.find('img', {'class':'c-profile__image'})['src']
TypeError: 'NoneType' object is not subscriptable

Exception in thread Thread-12:
Traceback (most recent call last):
  File "E:\software\anaconda\envs\gputest\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\software\anaconda\envs\gputest\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-26-dc3b696db679>", line 10, in get_image
    image = soup.find('img', {'class':'c-profile__image'})['src']
TypeError: 'NoneType' object is not subscriptable

Exception in thread Thread-9:
Traceback (most 

In [8]:
#water_mark_remover
import numpy as np
import cv2
import os
from tqdm import tqdm
DIR = 'E:/Youtube game/works/Ranchi/' #change
def remove_watermark(src):
    img = cv2.imread(src)
    mask = cv2.imread("E:/Youtube game/works/mask.jpg")
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    dst = cv2.inpaint(img,mask,3,cv2.INPAINT_TELEA)
    cv2.imwrite(DIR+'/Ranchi/'+src.split('/')[-1],dst) #change

for i in tqdm(os.listdir(DIR)):
    try:
        remove_watermark(DIR+i)
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████| 148/148 [00:03<00:00, 37.99it/s]


In [16]:
payload = 'sugarfree'
r = rp.post('http://127.0.0.1:8000/search/', json = payload)

In [17]:
m = json.loads(r.json())
print(len(m))
m

4


{'name': ['Sugar Free Gold Low Calorie Sweetener',
  'Sugar Free Natura Low Calorie Sweetener Powder',
  'Sugar Free Natura Low Calorie Sweetener Powder',
  'Sugar Free Green Stevia Pellets',
  'Sugar Free Natura Low Calorie Sweetener Pellets',
  'Sugar Free Gold Low Calorie Sweetener',
  'Sugar Free Gold Low Calorie Sweetener',
  'Sugar Free Gold Sachet',
  'Sugar Free Natura Low Calorie Sweetener Drop',
  'Sugar Free Green Powder',
  'Dabur Chyawanprakash Sugarfree',
  'Sugar Free Gold Low Calorie Sweetener',
  'Sugar Free Green Stevia Pellets',
  'Sugar Free Natura Low Calorie Sweetener Pellets',
  'Sugar Free Natura Low Calorie Sweetener Sachet',
  'Sugar Free Natura Low Calorie Sweetener Pellets',
  'Sugar Free Natura Low Calorie Sweetener',
  'Cremaffin Plus  Syrup Refreshing Sugar Free',
  'Mucinac 600 Effervescent Tablet Orange Sugar Free',
  'Neeri KFT  Sugar Free Syrup',
  'Depura Vitamin D3 60000IU Oral Solution Sugar Free',
  'Gelusil MPS Original Liquid Sugar Free Mint',
 

In [6]:
for i in ranchi_filter:
    print(i[-4], i[3])

https://www.practo.com/ranchi/doctor/arun-sarkar-general-physician?specialization=general physician 22644 Bihar Medical Council, 1986
https://www.practo.com/ranchi/doctor/dr-kunal-banka-dentist-1?specialization=dentist 19088A Karnataka State Dental Council, 2006
https://www.practo.com/ranchi/doctor/dr-pankaj-kumar-9-dentist?specialization=plastic surgeon 32830 Bihar Medical Council, 2001
https://www.practo.com/ranchi/doctor/dr-amitesh-anand-oncologist?specialization=oncologist 64977 Delhi Medical Council, 2013
https://www.practo.com/ranchi/doctor/lalit-kumar-kongari-ear-nose-throat-ent-specialist-1?specialization=ear-nose-throat (ent) specialist 17327 Delhi Medical Council, 2003
https://www.practo.com/ranchi/doctor/dr-aftab-alam-ansari-oncologist?specialization=oncologist 32940 Bihar Medical Council, 2001
https://www.practo.com/ranchi/doctor/dr-thounaojam-momon-singh-oncologist?specialization=oncologist 8749 Assam Medical Council, 1982
https://www.practo.com/ranchi/doctor/dr-rashmi-46-

In [2]:
model = tf.keras.models.load_model('gender_prediction_name.model')
token_table = {}
j = 0
for i in 'abcdefghijklmnopqrstuvwxyz':
    token_table[i] = j/25
    j+=1
def get_gender(name):
    try:
        name = [token_table[i.lower()] for i in list(name)]
        curr_len = len(name)
        padd_len = 15 - curr_len
        name += [-1.0]*padd_len
        if np.argmax(model.predict([name])) == 0:
            return('Female')
        else:
            return('Male')
    except:
        return('Male')

In [3]:
dat = rp.get('https://server.docplus.online/doctors/get').json()

In [4]:
for i in dat[:10]:
    print(i['firstName'], '-----------------', get_gender(i['firstName']))

Kunal ----------------- Male
Lalit ----------------- Male
Aftab ----------------- Male
Thounaojam ----------------- Male
Kumar ----------------- Male
Kumar ----------------- Male
Ratnesh ----------------- Female
Sarbjaya ----------------- Female
Nitisha ----------------- Female
Saha ----------------- Female


In [ ]:
def updateGender(_id, _gender):
    payload = {
        "id": _id,
        "gender": _gender
    }
    r = rp.post('https://server.docplus.online/doctors/profile/update', json = payload)
    if r.json()['status'] != True:
        print(r.json())
        
for i in tqdm(dat[4462:]):
    _gender = get_gender(i['firstName'])
    updateGender(i['_id'], _gender)

 97%|███████████████████████████████████████████████████████████████████████████▌  | 5464/5636 [36:07<01:11,  2.41it/s]

In [53]:
dat[4462]

{'_id': '607c7f2e34cb43518375a313',
 'firstName': 'Ashok',
 'lastName': 'Pansari'}